In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import os
import numpy as np
# import metview as mv

# Model

In [3]:
basePath_sfno = "/mnt/qb/work2/goswami0/gkd965/outputs"
file = "v10_SFNO_HalfYearForecast_startfrom_2022-01-01.nc"
# path = os.path.join(basePath, 'single_pressure_level', sfc)
print("loading sfno data")
# ds_sfno = xr.open_dataset(os.path.join(basePath_sfno,file), chunks={"step": 1})
ds_sfno = xr.open_dataset(os.path.join(basePath_sfno,file))
print("ds sfno: ")
print(ds_sfno.info())
print("fin loading")

loading sfno data
ds sfno: 
xarray.Dataset {
dimensions:
	step = 730 ;
	latitude = 721 ;
	longitude = 1440 ;

variables:
	datetime64[ns] time() ;
		time:long_name = initial time of forecast ;
		time:standard_name = forecast_reference_time ;
	timedelta64[ns] step(step) ;
		step:long_name = time since forecast_reference_time ;
		step:standard_name = forecast_period ;
	float64 heightAboveGround() ;
		heightAboveGround:long_name = height above the surface ;
		heightAboveGround:units = m ;
		heightAboveGround:positive = up ;
		heightAboveGround:standard_name = height ;
	float64 latitude(latitude) ;
		latitude:units = degrees_north ;
		latitude:standard_name = latitude ;
		latitude:long_name = latitude ;
		latitude:stored_direction = decreasing ;
	float64 longitude(longitude) ;
		longitude:units = degrees_east ;
		longitude:standard_name = longitude ;
		longitude:long_name = longitude ;
	datetime64[ns] valid_time(step) ;
		valid_time:standard_name = time ;
		valid_time:long_name = time ;
	fl

In [3]:
Model = ds_sfno.to_array(dim="v10").squeeze()

In [4]:
ds_sfno

<xarray.Dataset>
Dimensions:            (step: 730, latitude: 721, longitude: 1440)
Coordinates:
    time               datetime64[ns] ...
  * step               (step) timedelta64[ns] 0 days 06:00:00 ... 182 days 12...
    heightAboveGround  float64 ...
  * latitude           (latitude) float64 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude          (longitude) float64 0.0 0.25 0.5 ... 359.2 359.5 359.8
    valid_time         (step) datetime64[ns] ...
    surface            float64 ...
    meanSea            float64 ...
    entireAtmosphere   float64 ...
Data variables:
    v10                (step, latitude, longitude) float32 ...

In [5]:
from datetime import datetime, timedelta
t = np.arange(datetime(2021,1,1), datetime(2022,1,1), timedelta(hours=6)).astype(datetime)[:730]
x = [i.strftime("%m-%d %H") for i in t]
ds_sfno['hourofyear'] = np.array(x, dtype='object')
Model_swap = ds_sfno.swap_dims({'step': 'hourofyear'}) ############

ValueError: replacement dimension 'hourofyear' is not a 1D variable along the old dimension 'step'

In [6]:
ds_sfno

<xarray.Dataset>
Dimensions:            (step: 730, latitude: 721, longitude: 1440,
                        hourofyear: 730)
Coordinates:
    time               datetime64[ns] ...
  * step               (step) timedelta64[ns] 0 days 06:00:00 ... 182 days 12...
    heightAboveGround  float64 ...
  * latitude           (latitude) float64 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude          (longitude) float64 0.0 0.25 0.5 ... 359.2 359.5 359.8
    valid_time         (step) datetime64[ns] ...
    surface            float64 ...
    meanSea            float64 ...
    entireAtmosphere   float64 ...
  * hourofyear         (hourofyear) object '01-01 00' '01-01 06' ... '07-02 06'
Data variables:
    v10                (step, latitude, longitude) float32 ...

In [10]:
test = ds_sfno.reset_index("step")

In [9]:
test.isel(hourofyear='01-01 00')

TypeError: invalid indexer array, does not have integer dtype: array('01-01 00', dtype='<U8')

In [76]:
Model = ds_sfno["v10"]

# Reference

In [69]:
basePath = "/mnt/qb/goswami/data/era5"
# path = os.path.join(basePath, 'single_pressure_level', sfc)
print("loading data")
ds = xr.open_mfdataset(os.path.join(basePath, 'single_pressure_level', '10m_v_component_of_wind', "10m_v_component_of_wind_????.nc"),parallel=True)
print("ds: ")
print(ds.info())
print("fin loading")

loading data
ds: 
xarray.Dataset {
dimensions:
	longitude = 1440 ;
	latitude = 721 ;
	time = 368184 ;

variables:
	float32 longitude(longitude) ;
		longitude:units = degrees_east ;
		longitude:long_name = longitude ;
	float32 latitude(latitude) ;
		latitude:units = degrees_north ;
		latitude:long_name = latitude ;
	datetime64[ns] time(time) ;
		time:long_name = time ;
	float32 v10(time, latitude, longitude) ;
		v10:units = m s**-1 ;
		v10:long_name = 10 metre V wind component ;

// global attributes:
	:Conventions = CF-1.6 ;
	:history = 2022-07-14 14:01:09 GMT by grib_to_netcdf-2.25.1: /opt/ecmwf/mars-client/bin/grib_to_netcdf.bin -S param -o /cache/data1/adaptor.mars.internal-1657806762.467146-16033-8-e12dca0a-c8ec-405a-9f94-5781ea6dc0fe.nc /cache/tmp/e12dca0a-c8ec-405a-9f94-5781ea6dc0fe-adaptor.mars.internal-1657806279.235086-16033-9-tmp.grib ;
}None
fin loading


In [70]:
ds

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 368184)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 1979-01-01 ... 2021-12-31T23:00:00
Data variables:
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(8760, 721, 1440), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-07-14 14:01:09 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [71]:
from datetime import datetime, timedelta
import datetime as ddtime
steps = np.arange(datetime(1979,1,1), datetime(2022,1,1), timedelta(hours=1)).astype(datetime)
years = steps.astype('datetime64[Y]').astype(datetime)# + 1970
years_time = [datetime.combine(x, ddtime.time.min) for x in years]

In [93]:
steps[24*365 + 24*(31+27):24*365 + 24*(31+28)+2]

array([datetime.datetime(1980, 2, 28, 0, 0),
       datetime.datetime(1980, 2, 28, 1, 0),
       datetime.datetime(1980, 2, 28, 2, 0),
       datetime.datetime(1980, 2, 28, 3, 0),
       datetime.datetime(1980, 2, 28, 4, 0),
       datetime.datetime(1980, 2, 28, 5, 0),
       datetime.datetime(1980, 2, 28, 6, 0),
       datetime.datetime(1980, 2, 28, 7, 0),
       datetime.datetime(1980, 2, 28, 8, 0),
       datetime.datetime(1980, 2, 28, 9, 0),
       datetime.datetime(1980, 2, 28, 10, 0),
       datetime.datetime(1980, 2, 28, 11, 0),
       datetime.datetime(1980, 2, 28, 12, 0),
       datetime.datetime(1980, 2, 28, 13, 0),
       datetime.datetime(1980, 2, 28, 14, 0),
       datetime.datetime(1980, 2, 28, 15, 0),
       datetime.datetime(1980, 2, 28, 16, 0),
       datetime.datetime(1980, 2, 28, 17, 0),
       datetime.datetime(1980, 2, 28, 18, 0),
       datetime.datetime(1980, 2, 28, 19, 0),
       datetime.datetime(1980, 2, 28, 20, 0),
       datetime.datetime(1980, 2, 28, 21, 0)

In [73]:
fin = steps - years_time
fin[24*365+1]

datetime.timedelta(seconds=3600)

In [74]:
ffin = fin.astype('timedelta64[ns]')

In [80]:
ffin[300]

numpy.timedelta64(1080000000000000,'ns')

In [76]:
steps - years

TypeError: unsupported operand type(s) for -: 'datetime.datetime' and 'datetime.date'

In [88]:
ffin.size

376944

In [ ]:
# year_step = np.arange(datetime(1979,7,1), datetime(2022,1,1), timedelta(hours=1)).astype(datetime)d

In [87]:
# ds['hourofyear'] = xr.DataArray(ds.time.dt.strftime('%m-%d %H'), coords=ds.time.coords)
# ds['step'] = np.array([21600000000000*i for i in range(720)], dtype='timedelta64[ns]')
ds['step'] = xr.DataArray(fin, coords=ds.time.coords)
# ds['step'] = ffin
# ds_hourofyear = ds.groupby("hourofyear")
ds_hourofyear = ds.groupby("step")
mean = ds_hourofyear.mean()

ValueError: conflicting sizes for dimension 'time': length 376944 on the data but length 368184 on coordinate 'time'

In [82]:
mean.info()

xarray.Dataset {
dimensions:
	longitude = 1440 ;
	latitude = 721 ;
	time = 368184 ;
	step = 8784 ;

variables:
	float32 longitude(longitude) ;
		longitude:units = degrees_east ;
		longitude:long_name = longitude ;
	float32 latitude(latitude) ;
		latitude:units = degrees_north ;
		latitude:long_name = latitude ;
	datetime64[ns] time(time) ;
		time:long_name = time ;
	float32 v10(step, time, latitude, longitude) ;
		v10:units = m s**-1 ;
		v10:long_name = 10 metre V wind component ;
	timedelta64[ns] step(step) ;

// global attributes:
	:Conventions = CF-1.6 ;
	:history = 2022-07-14 14:01:09 GMT by grib_to_netcdf-2.25.1: /opt/ecmwf/mars-client/bin/grib_to_netcdf.bin -S param -o /cache/data1/adaptor.mars.internal-1657806762.467146-16033-8-e12dca0a-c8ec-405a-9f94-5781ea6dc0fe.nc /cache/tmp/e12dca0a-c8ec-405a-9f94-5781ea6dc0fe-adaptor.mars.internal-1657806279.235086-16033-9-tmp.grib ;
}

In [86]:
mean.drop_dims("time")

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, step: 8784)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * step       (step) timedelta64[ns] 00:00:00 01:00:00 ... 365 days 23:00:00
Data variables:
    *empty*
Attributes:
    Conventions:  CF-1.6
    history:      2022-07-14 14:01:09 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [16]:
mean.isel(hourofyear='01-01 00')

TypeError: invalid indexer array, does not have integer dtype: array('01-01 00', dtype='<U8')

In [57]:
Ref = mean["v10"]

In [58]:
Ref

<xarray.DataArray 'v10' (hourofyear: 8784, latitude: 721, longitude: 1440)>
dask.array<concatenate, shape=(8784, 721, 1440), dtype=float32, chunksize=(1, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * longitude   (longitude) float32 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * latitude    (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * hourofyear  (hourofyear) object '01-01 00' '01-01 01' ... '12-31 23'
Attributes:
    units:      m s**-1
    long_name:  10 metre V wind component

In [7]:
#Ref = mean.to_array(dim='v10').squeeze() ############# like that or with ["v10"] ?

In [59]:
# hourofyear for leapyear
start_feb_29 = 24*(31+28)
end_feb_29 = start_feb_29 + 24
end_feb_29 - start_feb_29

Ref_noleapyear = xr.concat((Ref[:start_feb_29,:,:], Ref[end_feb_29:,:,:]),dim='hourofyear')

In [60]:
Ref_noleapyear[start_feb_29 - 1]

<xarray.DataArray 'v10' (latitude: 721, longitude: 1440)>
dask.array<getitem, shape=(721, 1440), dtype=float32, chunksize=(721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * longitude   (longitude) float32 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * latitude    (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
    hourofyear  <U8 '02-28 23'
Attributes:
    units:      m s**-1
    long_name:  10 metre V wind component

In [61]:
Ref_6hr = Ref_noleapyear[::6,:,:]
Ref_6hr = Ref_6hr[:730,:,:]
Ref_6hr.shape

(730, 721, 1440)

In [62]:
Ref_6hr

<xarray.DataArray 'v10' (hourofyear: 730, latitude: 721, longitude: 1440)>
dask.array<getitem, shape=(730, 721, 1440), dtype=float32, chunksize=(1, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * longitude   (longitude) float32 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * latitude    (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * hourofyear  (hourofyear) object '01-01 00' '01-01 06' ... '07-02 06'
Attributes:
    units:      m s**-1
    long_name:  10 metre V wind component

In [63]:
Ref_6hr_d = Ref_6hr.to_dataset()

In [64]:
Ref_6hr_d['step'] = np.array([21600000000000*i for i in range(730)], dtype='timedelta64[ns]')

In [65]:
Ref_6hr_d

<xarray.Dataset>
Dimensions:     (longitude: 1440, latitude: 721, hourofyear: 730, step: 730)
Coordinates:
  * longitude   (longitude) float32 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * latitude    (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * hourofyear  (hourofyear) object '01-01 00' '01-01 06' ... '07-02 06'
  * step        (step) timedelta64[ns] 0 days 00:00:00 ... 182 days 06:00:00
Data variables:
    v10         (hourofyear, latitude, longitude) float32 dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>

In [68]:
Ref_6hr_dd = Ref_6hr_d.swap_dims({'hourofyear': 'step'}) ############

ValueError: replacement dimension 'step' is not a 1D variable along the old dimension 'hourofyear'

In [41]:
Ref_6hr = Ref_6hr.rename({"hourofyear":"step"})
# np.array([21600000000000*i for i in range(20)], dtype='timedelta64[ns]')

In [42]:
Ref_6hr

<xarray.DataArray 'v10' (step: 730, latitude: 721, longitude: 1440)>
dask.array<getitem, shape=(730, 721, 1440), dtype=float32, chunksize=(1, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * step       (step) object '01-01 00' '01-01 06' ... '07-02 00' '07-02 06'
Attributes:
    units:      m s**-1
    long_name:  10 metre V wind component

In [79]:
from datetime import datetime, timedelta
t = np.arange(datetime(2021,1,1), datetime(2022,1,1), timedelta(hours=6)).astype(datetime)[:730]
x = [i.strftime("%m-%d %H") for i in t]
t['hourofyear'] = np.array(x, dtype='object')
Model_swap = t.swap_dims({'step': 'hourofyear'}) ############

In [82]:
t

<xarray.Dataset>
Dimensions:            (step: 730, latitude: 721, longitude: 1440,
                        hourofyear: 730)
Coordinates:
    time               datetime64[ns] ...
  * step               (step) timedelta64[ns] 0 days 06:00:00 ... 182 days 12...
    heightAboveGround  float64 ...
  * latitude           (latitude) float64 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude          (longitude) float64 0.0 0.25 0.5 ... 359.2 359.5 359.8
    valid_time         (step) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    surface            float64 ...
    meanSea            float64 ...
    entireAtmosphere   float64 ...
  * hourofyear         (hourofyear) object '01-01 00' '01-01 06' ... '07-02 06'
Data variables:
    v10                (step, latitude, longitude) float32 dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>

In [83]:
t = Model.to_dataset()
t['hourofyear'] = np.array(x, dtype='object')
Model_swap = t.swap_dims({'step': 'hourofyear'}) ############

ValueError: replacement dimension 'hourofyear' is not a 1D variable along the old dimension 'step'

In [44]:
RMSE = np.divide(Model,Ref_6hr)

In [45]:
RMSE

<xarray.DataArray (step: 0, latitude: 721, longitude: 1440)>
dask.array<divide, shape=(0, 721, 1440), dtype=float32, chunksize=(0, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * step               (step) object 
  * latitude           (latitude) float64 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude          (longitude) float64 0.0 0.25 0.5 ... 359.2 359.5 359.8
    time               datetime64[ns] ...
    heightAboveGround  float64 ...
    valid_time         (step) datetime64[ns] dask.array<chunksize=(0,), meta=np.ndarray>
    surface            float64 ...
    meanSea            float64 ...
    entireAtmosphere   float64 ...
    v10                <U3 'v10'

In [ ]:
RMSE = 1 - Model.to_numpy() / (Ref_6hr[:730,:,:]).to_numpy()

<xarray.DataArray (step: 730, latitude: 721, longitude: 1440, hourofyear: 730)>
dask.array<divide, shape=(730, 721, 1440, 730), dtype=float32, chunksize=(1, 721, 1440, 1), chunktype=numpy.ndarray>
Coordinates:
    time               datetime64[ns] ...
  * step               (step) timedelta64[ns] 0 days 06:00:00 ... 182 days 12...
    heightAboveGround  float64 ...
  * latitude           (latitude) float64 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude          (longitude) float64 0.0 0.25 0.5 ... 359.2 359.5 359.8
    valid_time         (step) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    surface            float64 ...
    meanSea            float64 ...
    entireAtmosphere   float64 ...
    v10                <U3 'v10'
  * hourofyear         (hourofyear) object '01-01 00' '01-01 06' ... '07-02 06'

In [ ]:
t = Model[0,:,:].to_numpy()/Ref_6hr[0,:,:].to_numpy()

In [21]:
t[0,0]

<xarray.DataArray ()>
dask.array<getitem, shape=(), dtype=float32, chunksize=(), chunktype=numpy.ndarray>
Coordinates:
    time               datetime64[ns] ...
    step               timedelta64[ns] 06:00:00
    heightAboveGround  float64 ...
    latitude           float64 90.0
    longitude          float64 0.0
    valid_time         datetime64[ns] dask.array<chunksize=(), meta=np.ndarray>
    surface            float64 ...
    meanSea            float64 ...
    entireAtmosphere   float64 ...
    v10                <U3 'v10'
    hourofyear         <U8 '01-01 00'

In [23]:
Model[0,0,0].to_numpy()

array(0.21703911, dtype=float32)

In [21]:
type(Model)

xarray.core.dataarray.DataArray

In [22]:
type(Ref)

xarray.core.dataarray.DataArray